In [1]:
import pandas as pd

Checking an example csv for heart rate


In [2]:
df_ex_hr=pd.read_csv('datasets/COVID-19-Wearables/A0KX894_hr.csv')
df_ex_hr.head(10)

,user,datetime,heartrate
0,A0KX894,2026-07-22 00:00:04,78
1,A0KX894,2026-07-22 00:00:09,77
2,A0KX894,2026-07-22 00:00:14,76
3,A0KX894,2026-07-22 00:00:19,75
4,A0KX894,2026-07-22 00:00:24,74
5,A0KX894,2026-07-22 00:00:29,75
6,A0KX894,2026-07-22 00:00:34,74
7,A0KX894,2026-07-22 00:00:49,74
8,A0KX894,2026-07-22 00:00:54,77
9,A0KX894,2026-07-22 00:00:59,76


In [3]:
df_ex_hr.describe()

,heartrate
count,879561.000000
mean,76.706986
std,14.771697
min,47.000000
25%,66.000000
50%,75.000000
75%,85.000000
max,181.000000


In [4]:
df_ex_hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879561 entries, 0 to 879560
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user       879561 non-null  object
 1   datetime   879561 non-null  object
 2   heartrate  879561 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 20.1+ MB


In [5]:
df_ex_hr['datetime'].describe()

count                  879561
unique                 879561
top       2026-07-22 00:00:04
freq                        1
Name: datetime, dtype: object

Max heart rate is normally 220 - 20, as the data recorded complies with the standards of Stanford universities, the participants should be around or superior to 20. lets do a quick check on one of the recorded data


In [6]:
nan_entries = df_ex_hr.isna().sum()
illogical_entries = df_ex_hr[(df_ex_hr['heartrate'] < 30)| (df_ex_hr['heartrate'] > 200)].sum()
print('Number of NaN entries: ', nan_entries)
print('Number of illogical entries: ', illogical_entries)

Number of NaN entries:  user         0
datetime     0
heartrate    0
dtype: int64
Number of illogical entries:  user         0.0
datetime     0.0
heartrate    0.0
dtype: float64


Lets ittereate over the directories and save a dictionary containing the users name and the corresponding hr and steps

In [7]:
import os
dir_csv = 'datasets/COVID-19-Wearables/'
files = [f for f in os.listdir(dir_csv) if f.endswith('.csv')] # we can straight do it with os.listdir
# noticed there are some files with user name containing_1 or _2, so we need to put that into consideration
#user_name = [f.split('_')[0] if f.count("_")==1 else ""for f in files]
# will deal with it inside the dictionary better
user_name = [f.split('_')[0] for f in files]
mult=0
for i in user_name:
    if(user_name.count(i)) < 1:
        print('missing data for user: ', i)
    if(user_name.count(i)) > 2:
        mult+=1
       
user_name_unique = list(set(user_name))

dict_user_data={}   
for i in user_name_unique:
    dict_user_data[i] = dict()# I am not concatinating strings, there is a user with multi entries
    dict_user_data[i]['hr'] = [f for f in files if f == i+'_hr.csv']
   
    
# to visualize all the directory structure


In [8]:
merged_df_same_user_dict = {}
merged_df_same_user_dict_2 = {}

for user in dict_user_data.keys():
    if dict_user_data[user]['hr'] != []:
        df_hr = pd.read_csv(dir_csv + dict_user_data[user]['hr'][0])
        df_hr['datetime'] = pd.to_datetime(df_hr['datetime'])
        df_hr.set_index('datetime', inplace=True)
        df_hr.resample('5Min').mean() # resample to 1 min
        #df_hr['heartrate'].resample('1min').mean()
        df_hr['datetime'] = df_hr.index
    merged_df_same_user_dict[user]=df_hr
    #merged_df_same_user_dict_2[user]=pd.merge(df_hr, df_steps, left_index=True, right_index=True, how="outer")
    



In [ ]:
df_t = merged_df_same_user_dict['A0KX894']

In [ ]:
df_t.head()

,user,heartrate,datetime
datetime,,,
2026-07-22 00:00:04,A0KX894,78,2026-07-22 00:00:04
2026-07-22 00:00:09,A0KX894,77,2026-07-22 00:00:09
2026-07-22 00:00:14,A0KX894,76,2026-07-22 00:00:14
2026-07-22 00:00:19,A0KX894,75,2026-07-22 00:00:19
2026-07-22 00:00:24,A0KX894,74,2026-07-22 00:00:24


In [ ]:
df_t.head()

,user,heartrate,datetime
datetime,,,
2026-07-22 00:00:04,A0KX894,78,2026-07-22 00:00:04
2026-07-22 00:00:09,A0KX894,77,2026-07-22 00:00:09
2026-07-22 00:00:14,A0KX894,76,2026-07-22 00:00:14
2026-07-22 00:00:19,A0KX894,75,2026-07-22 00:00:19
2026-07-22 00:00:24,A0KX894,74,2026-07-22 00:00:24


In [ ]:
len(df_t)

In [ ]:
df_master = pd.concat(merged_df_same_user_dict.values(), ignore_index=True)

In [ ]:
df_master.head()

,user,heartrate,datetime
0,A2XFW2N,79,2025-05-07 23:00:00
1,A2XFW2N,67,2025-05-07 23:01:00
2,A2XFW2N,63,2025-05-07 23:02:00
3,A2XFW2N,63,2025-05-07 23:03:00
4,A2XFW2N,63,2025-05-07 23:04:00


In [ ]:
df_master["user"].unique() # confirmed all users are in the master df

array(['A2XFW2N', 'AFVAEC7', 'AX6281V', 'AF3J1YC', 'AR4FPCC', 'A65HVGP',
       'AKV66US', 'A99ZKKW', 'AJMQUVV', 'A3OU183', 'AUY8KYW', 'AZIK4ZA',
       'A17YCA2', 'AQC0L71', 'AGA8XUN', 'ARR2IKE', 'A45F9E6', 'AIFDJZB',
       'AAGTWZK', 'ATHKM6V', 'A0NVTRV', 'AA2KP1S', 'AUCGUF3', 'AQR8ZSS',
       'ATHBFWX', 'APDJ1QP', 'AWA2KJK', 'AQ4TMLV', 'AX3KEW9', 'ALKAXMZ',
       'A2D7K4A', 'AYVQUF1', 'AD77K91', 'A4E0D03', 'A35BJNV', 'A11V1FH',
       'AFPB8J2', 'AWRBUQZ', 'A0KX894', 'AKXN5ZZ', 'AJWW3IY', 'A0N9NV4',
       'AF3MXM1', 'AKTGD8X', 'AE0MQ94', 'AOS4BSJ', 'AOGFRXL', 'ATT9RR1',
       'AAXAA7Z', 'A2P3LTM', 'APGIB2T', 'ALZDAVZ', 'A0L9BM2', 'AOB9SON',
       'AK7YRBU', 'AJ7TSV9', 'AOYM4KG', 'AF8R0I6', 'A91HEZV', 'AZ35PI5',
       'AMQUHOQ', 'AEOBCFJ', 'A0VFT1N', 'ASFODQR', 'AUILKHG', 'A0822M0',
       'AURCTAK', 'AXD3W8O', 'AJ0DKQ3', 'AYEFCWQ', 'A8CBEJZ', 'AEOHH30',
       'A7EM0B6', 'AL48GP3', 'A7EAWA7', 'AFHOHOM', 'AXCO7I9', 'A3ADWUT',
       'AL3KT5B', 'A6GEBIK', 'ARFYLMK', 'AQ25Y0L', 

In [ ]:
df_master.to_csv('datasets/df_master_hr_light.csv')